In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus
from konlpy.tag import Okt
from collections import Counter

In [2]:
okt = Okt()

In [3]:
pd.set_option('display.max_columns', None)

# Skirt 리뷰 전처리

In [4]:
df_skirt = pd.read_csv('./musinsa_review_skirt.csv')

In [5]:
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [6]:
df_skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12391 entries, 0 to 12390
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       12375 non-null  object
 1   date           12391 non-null  object
 2   goodsNo        12391 non-null  int64 
 3   userSexMen     12391 non-null  int64 
 4   userSexWomen   12391 non-null  int64 
 5   userHeight     12391 non-null  object
 6   userWeight     12391 non-null  object
 7   goodsSize      12391 non-null  object
 8   reviewContent  12391 non-null  object
 9   reviewImg      3049 non-null   object
 10  reviewStyle    12391 non-null  int64 
 11  size           12391 non-null  int64 
 12  bright         12391 non-null  int64 
 13  color          12391 non-null  int64 
 14  thickness      12391 non-null  int64 
 15  weightness     12391 non-null  int64 
 16  touch          12391 non-null  int64 
 17  helpNo         12391 non-null  int64 
 18  styleLikeNo    12391 non-n

In [7]:
df_skirt.shape

(12391, 19)

### 탈퇴 인원 제거

In [8]:
df_skirt.dropna(subset=["userName"], axis=0, inplace=True)
df_skirt = df_skirt.reset_index(drop=True)

In [9]:
df_skirt.shape

(12375, 19)

### Date를 년, 월, 일로 등분

In [10]:
df_skirt.dropna(subset=["date"], axis=0, inplace=True)

In [11]:
df_skirt.shape

(12375, 19)

In [12]:
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


### goodsNo

In [13]:
df_skirt.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [14]:
df_skirt.shape

(12375, 19)

### 성별

In [15]:
df_skirt[df_skirt.userSexMen == 1].shape

(192, 19)

In [16]:
df_skirt[df_skirt.userSexWomen==1].shape

(12183, 19)

In [17]:
192+12183

12375

### 키와 몸무게

In [18]:
df_skirt.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [19]:
df_skirt.shape

(12375, 19)

In [20]:
df_skirt.userHeight = df_skirt.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_skirt.userHeight = df_skirt.userHeight.astype('int64')

In [21]:
df_skirt.userWeight = df_skirt.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_skirt.userWeight = df_skirt.userWeight.astype('int64')

In [22]:
a = list((df_skirt[df_skirt.userHeight==0][df_skirt.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\3390044186.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_skirt[df_skirt.userHeight==0][df_skirt.userWeight==0]).index)


[]

In [23]:
df_skirt = df_skirt.drop(a)
df_skirt.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022.03.06,2340070,0,1,160,48,M,"['보통 여기 치마는 m 사면 무난하게 맞았는데', <br/>, '얘는 좀 허리가 ...",NaN,0,1,2,2,0,0,0,0,0
12371,얼래프,2022.03.03,2340070,0,1,163,47,S,['패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ'],NaN,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022.02.25,2340070,0,1,160,50,S,"['엉덩이 있는 편인데 S 잘맞아요', <br/>, '허리는 25-26 정도인데 적...",NaN,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022.02.18,2340070,0,1,158,50,M,['실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박'],NaN,0,2,2,1,0,0,0,0,0


### 이미지

In [24]:
df_skirt.reviewStyle.value_counts()

0    9328
1    3047
Name: reviewStyle, dtype: int64

In [25]:
df_skirt.reviewImg = df_skirt.reviewImg.fillna("-")

In [26]:
df_skirt.reviewImg

0        https://image.msscdn.net/data/estimate/2029080...
1                                                        -
2                                                        -
3        https://image.msscdn.net/data/estimate/1911322...
4        https://image.msscdn.net/data/estimate/1911322...
                               ...                        
12370                                                    -
12371                                                    -
12372                                                    -
12373                                                    -
12374                                                    -
Name: reviewImg, Length: 12375, dtype: object

In [27]:
type(df_skirt.reviewImg[1])

str

### Contents

In [28]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1].split(","))

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
["['생각보다 짧은 치마'", ' <br/>', " '제 체격에는 좀 작게 느껴질만한 사이즈'", ' <br/>', " '디자인은 되게 예쁨']"]


In [29]:
cnt = 0
result = []
for i in df_skirt.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

2


In [30]:
df_skirt[df_skirt.reviewContent=='[\'데님 색이 너무 맘에 들어서 구매했습니다!\', <br/>, \'키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\n<div class="review-content-photo__wrap">\n<ul class="review-content-photo__list">\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.list\');">\n<img alt="밀리언코르(MILLIONCOR) [DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이) 후기" src="//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.view"/>\n</li>\n</ul>\n</div>\n</div>, \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">[DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이)</span>, \'\\n\']']

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
1184,여신예리,2021.04.19,1618698,0,1,152,53,M,"['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가...",https://image.msscdn.net/data/estimate/1618698...,1,2,2,2,0,0,0,0,0


In [31]:
df_skirt[df_skirt.reviewContent=='[\'허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">Inside-Out Kangaroo Skirt Melange-Reverse</span>, \'\\n\']']

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
11323,써드3,2021.07.23,1788798,0,1,163,50,S,['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같...,-,0,2,2,2,0,0,0,0,0


In [32]:
df_skirt.loc[1186, 'reviewContent'] = result[0].split('</div')[0][:]
df_skirt.loc[11338, 'reviewContent'] = result[1].split('</div')[0][:]

In [33]:
print(df_skirt.loc[1186].reviewContent)
print(df_skirt.loc[11338].reviewContent)

['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_', '!
['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>', '!


In [34]:
df_skirt.reviewContent = df_skirt.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [35]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1])

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
['생각보다 짧은 치마', '제 체격에는 좀 작게 느껴질만한 사이즈', '디자인은 되게 예쁨']


In [36]:
type(df_skirt.reviewContent.loc[0])

list

### 사이즈

In [37]:
sub = df_skirt[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', sub.index[:], True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [38]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [39]:
for i in range(2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,2029080,0
1,1911322,M,1911322,1
2,1911322,S,1911322,2
3,1625877,FREE,1625877,3
4,1586158,L(28-30),1586158,4
...,...,...,...,...
2649,2117256,S,2117256,2649
2650,2117256,M,2117256,2650
2651,2117256,L,2117256,2651
2652,2340070,M,2340070,2652


In [40]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2654 entries, 0 to 2653
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2654 non-null   object
 1   goodsSize   2654 non-null   object
 2   newGoodsNo  2654 non-null   object
 3   index       2654 non-null   object
dtypes: object(4)
memory usage: 168.2+ KB


In [41]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,20290800,0
1,1911322,M,19113221,1
2,1911322,S,19113222,2
3,1625877,FREE,16258773,3
4,1586158,L(28-30),15861584,4
...,...,...,...,...
2649,2117256,S,21172562649,2649
2650,2117256,M,21172562650,2650
2651,2117256,L,21172562651,2651
2652,2340070,M,23400702652,2652


In [42]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [43]:
df_skirt

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022.03.06,2340070,0,1,160,48,M,"[보통 여기 치마는 m 사면 무난하게 맞았는데, 얘는 좀 허리가 크네요, 그래도 상...",-,0,1,2,2,0,0,0,0,0
12371,얼래프,2022.03.03,2340070,0,1,163,47,S,[패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ],-,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022.02.25,2340070,0,1,160,50,S,"[엉덩이 있는 편인데 S 잘맞아요, 허리는 25-26 정도인데 적당히 맞는편]",-,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022.02.18,2340070,0,1,158,50,M,[실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박],-,0,2,2,1,0,0,0,0,0


In [44]:
df_skirt = pd.merge(left=df_skirt, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,k_hnnj,2021.10.14,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0,20290800
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,0,0,0,0,0,20290800
2,k_hnnj,2021.08.12,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,0,0,0,0,0,20290800
3,미역이233,2021.08.06,1911322,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113221
4,블리츠크랭크!,2021.07.07,1911322,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113222


In [45]:
df_skirt.shape

(12375, 20)

In [46]:
df_skirt = df_skirt.drop(['thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'], axis=1)

In [47]:
df_skirt

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,newGoodsNo
0,k_hnnj,2021.10.14,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,20290800
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,20290800
2,k_hnnj,2021.08.12,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,20290800
3,미역이233,2021.08.06,1911322,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,19113221
4,블리츠크랭크!,2021.07.07,1911322,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,19113222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022.03.06,2340070,0,1,160,48,M,"[보통 여기 치마는 m 사면 무난하게 맞았는데, 얘는 좀 허리가 크네요, 그래도 상...",-,0,1,2,2,23400702652
12371,얼래프,2022.03.03,2340070,0,1,163,47,S,[패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ],-,0,1,2,2,23400702653
12372,교촌한입만,2022.02.25,2340070,0,1,160,50,S,"[엉덩이 있는 편인데 S 잘맞아요, 허리는 25-26 정도인데 적당히 맞는편]",-,0,2,1,1,23400702653
12373,야옹웅,2022.02.18,2340070,0,1,158,50,M,[실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박],-,0,2,2,1,23400702652


In [48]:
df_skirt.insert(0, 'reviewId', 0, True)

In [49]:
df_skirt.reviewId = range(len(df_skirt))

In [50]:
df_skirt

,reviewId,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,newGoodsNo
0,0,k_hnnj,2021.10.14,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,20290800
1,1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,20290800
2,2,k_hnnj,2021.08.12,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,20290800
3,3,미역이233,2021.08.06,1911322,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,19113221
4,4,블리츠크랭크!,2021.07.07,1911322,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,19113222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,12370,뉴비_54bed8fe,2022.03.06,2340070,0,1,160,48,M,"[보통 여기 치마는 m 사면 무난하게 맞았는데, 얘는 좀 허리가 크네요, 그래도 상...",-,0,1,2,2,23400702652
12371,12371,얼래프,2022.03.03,2340070,0,1,163,47,S,[패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ],-,0,1,2,2,23400702653
12372,12372,교촌한입만,2022.02.25,2340070,0,1,160,50,S,"[엉덩이 있는 편인데 S 잘맞아요, 허리는 25-26 정도인데 적당히 맞는편]",-,0,2,1,1,23400702653
12373,12373,야옹웅,2022.02.18,2340070,0,1,158,50,M,[실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박],-,0,2,2,1,23400702652


In [51]:
df_skirt.to_json('review_skirt.json', orient='records', force_ascii=False)

# Onepiece 리뷰 전처리

In [93]:
df_onepiece = pd.read_csv('./musinsa_review_onepiece.csv')

In [94]:
df_onepiece.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


In [95]:
df_onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8461 entries, 0 to 8460
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       8448 non-null   object
 1   date           8461 non-null   object
 2   goodsNo        8461 non-null   int64 
 3   userSexMen     8461 non-null   int64 
 4   userSexWomen   8461 non-null   int64 
 5   userHeight     8461 non-null   object
 6   userWeight     8461 non-null   object
 7   goodsSize      8461 non-null   object
 8   reviewContent  8461 non-null   object
 9   reviewImg      2117 non-null   object
 10  reviewStyle    8461 non-null   int64 
 11  size           8461 non-null   int64 
 12  bright         8461 non-null   int64 
 13  color          8461 non-null   int64 
 14  thickness      8461 non-null   int64 
 15  weightness     8461 non-null   int64 
 16  touch          8461 non-null   int64 
 17  helpNo         8461 non-null   int64 
 18  styleLikeNo    8461 non-null

In [96]:
df_onepiece.shape

(8461, 19)

### 탈퇴 인원 제거

In [97]:
df_onepiece.dropna(subset=["userName"], axis=0, inplace=True)
df_onepiece = df_onepiece.reset_index(drop=True)

In [98]:
df_onepiece.shape

(8448, 19)

### Date를 년, 월, 일로 등분

In [99]:
df_onepiece.dropna(subset=["date"], axis=0, inplace=True)

In [100]:
df_onepiece.shape

(8448, 19)

In [101]:
df_onepiece.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


### goodsNo

In [102]:
df_onepiece.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [103]:
df_onepiece.shape

(8448, 19)

### 성별

In [104]:
df_onepiece[df_onepiece.userSexMen == 1].shape

(144, 19)

In [105]:
df_onepiece[df_onepiece.userSexWomen==1].shape

(8304, 19)

In [106]:
144+8304

8448

### 키와 몸무게

In [107]:
df_onepiece.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [108]:
df_onepiece.shape

(8448, 19)

In [109]:
df_onepiece.userHeight = df_onepiece.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_onepiece.userHeight = df_onepiece.userHeight.astype('int64')

In [110]:
df_onepiece.userWeight = df_onepiece.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_onepiece.userWeight = df_onepiece.userWeight.astype('int64')

In [111]:
a = list((df_onepiece[df_onepiece.userHeight==0][df_onepiece.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\1381632839.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_onepiece[df_onepiece.userHeight==0][df_onepiece.userWeight==0]).index)


[5919]

In [112]:
df_onepiece = df_onepiece.drop(a)
df_onepiece.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,1933622,0,1,163,53,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,1933622,0,1,163,53,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2297184,0,1,152,58,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442,yu.r,2020.09.04,1440679,0,1,163,52,NONE,['패턴이 너무 이뻐서 구매했는데 생각보다 치마길이는 짧아서 너무 아쉬웠어요 ㅠㅠ ...,NaN,0,3,2,2,0,0,0,0,0
8443,뚝서,2021.08.10,1941295,0,1,160,50,PINK/FREE,"['원피스는 통통하신 분들도 잘 들어갈 정도로 밴딩이 잘 되어 있어요', <br/>...",NaN,0,2,2,1,0,0,0,0,0
8444,뉴비_ed629198,2021.09.04,1464994,0,1,160,48,FREE,['정말정말 만족하는 원피스 입니다 생각한것 보다 훨씬 예쁘고 봄여름가을 상관없이 ...,https://image.msscdn.net/data/estimate/1464994...,1,2,2,2,0,0,0,0,0
8445,소영파크,2022.02.09,1464994,0,1,170,60,FREE,['여리여리해보이고 샤랄라하고 사이즈 프리지만 길이감도 적당'],NaN,0,2,2,2,0,0,0,0,0


### 이미지

In [113]:
df_onepiece.reviewStyle.value_counts()

0    6331
1    2116
Name: reviewStyle, dtype: int64

In [114]:
df_onepiece.reviewImg = df_onepiece.reviewImg.fillna("-")

In [115]:
df_onepiece.reviewImg

0       https://image.msscdn.net/data/estimate/1933622...
1                                                       -
2       https://image.msscdn.net/data/estimate/2297184...
3                                                       -
4                                                       -
                              ...                        
8443                                                    -
8444                                                    -
8445    https://image.msscdn.net/data/estimate/1464994...
8446                                                    -
8447                                                    -
Name: reviewImg, Length: 8447, dtype: object

### Contents

In [116]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1].split(","))

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
["['여름에 편하게 입기 좋을것 같아요!'", ' <br/>', " '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ'", ' <br/>', " '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']"]


In [117]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_onepiece.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

0


In [118]:
df_onepiece.reviewContent = df_onepiece.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [119]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1])

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
['여름에 편하게 입기 좋을것 같아요!', '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ', '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']


### 사이즈

In [120]:
sub = df_onepiece[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))
sub['index'] = range(2654, 2654+len(sub))
sub.index = range(2654, 2654+len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [121]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [122]:
len(sub)

2047

In [123]:
for i in range(2654, len(sub)+2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [124]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 2654 to 4700
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2047 non-null   object
 1   goodsSize   2047 non-null   object
 2   newGoodsNo  2047 non-null   object
 3   index       2047 non-null   object
dtypes: object(4)
memory usage: 64.1+ KB


In [125]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,19336222654,2654
2655,2297184,M,22971842655,2655
2656,2297184,S,22971842656,2656
2657,2297184,XS,22971842657,2657
2658,1771393,MINT/FREE,17713932658,2658
...,...,...,...,...
4696,1782233,M,17822334696,4696
4697,1790851,M,17908514697,4697
4698,1440679,NONE,14406794698,4698
4699,1941295,PINK/FREE,19412954699,4699


In [126]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [127]:
df_onepiece = pd.merge(left=df_onepiece, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_onepiece.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,odddr,2021.05.29,1933622,0,1,163,53,none,[다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0,19336222654
1,odddr,2021.05.29,1933622,0,1,163,53,none,"[여름에 편하게 입기 좋을것 같아요!, 생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ, 앞...",-,0,1,2,2,0,0,0,0,0,19336222654
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"[길이감 좋고 스타일 꾸안꾸느낌, 빨리 날 좋아지면 입고싶음, 넉넉하게 나왔으나 퍼...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0,22971842655
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"[여유있게 입으려고 M사이즈 구매, 나그랑이라 어깨 안넓어보이고 좋음, 트임이 생각...",-,0,2,2,2,0,0,0,0,0,22971842655
4,YnSUGA,2022.02.22,2297184,0,1,152,58,S,[봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 것...,-,0,2,2,2,0,0,0,0,0,22971842656


In [128]:
df_onepiece.shape

(8447, 20)

In [129]:
df_onepiece.insert(0, 'reviewId', 0, True)

In [132]:
df_onepiece.reviewId = range(12375, 12375 + len(df_onepiece))

In [133]:
df_onepiece

,reviewId,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,12375,odddr,2021.05.29,1933622,0,1,163,53,none,[다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0,19336222654
1,12376,odddr,2021.05.29,1933622,0,1,163,53,none,"[여름에 편하게 입기 좋을것 같아요!, 생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ, 앞...",-,0,1,2,2,0,0,0,0,0,19336222654
2,12377,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"[길이감 좋고 스타일 꾸안꾸느낌, 빨리 날 좋아지면 입고싶음, 넉넉하게 나왔으나 퍼...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0,22971842655
3,12378,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160,55,M,"[여유있게 입으려고 M사이즈 구매, 나그랑이라 어깨 안넓어보이고 좋음, 트임이 생각...",-,0,2,2,2,0,0,0,0,0,22971842655
4,12379,YnSUGA,2022.02.22,2297184,0,1,152,58,S,[봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 것...,-,0,2,2,2,0,0,0,0,0,22971842656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442,20817,yu.r,2020.09.04,1440679,0,1,163,52,NONE,[패턴이 너무 이뻐서 구매했는데 생각보다 치마길이는 짧아서 너무 아쉬웠어요 ㅠㅠ 근...,-,0,3,2,2,0,0,0,0,0,14406794698
8443,20818,뚝서,2021.08.10,1941295,0,1,160,50,PINK/FREE,"[원피스는 통통하신 분들도 잘 들어갈 정도로 밴딩이 잘 되어 있어요, 다만 입고 난...",-,0,2,2,1,0,0,0,0,0,19412954699
8444,20819,뉴비_ed629198,2021.09.04,1464994,0,1,160,48,FREE,[정말정말 만족하는 원피스 입니다 생각한것 보다 훨씬 예쁘고 봄여름가을 상관없이 다...,https://image.msscdn.net/data/estimate/1464994...,1,2,2,2,0,0,0,0,0,14649944700
8445,20820,소영파크,2022.02.09,1464994,0,1,170,60,FREE,[여리여리해보이고 샤랄라하고 사이즈 프리지만 길이감도 적당],-,0,2,2,2,0,0,0,0,0,14649944700


In [134]:
df_onepiece.to_json('review_onepiece.json', orient='records', force_ascii=False)

# Pants 리뷰 전처리

In [135]:
df_pants1 = pd.read_csv('./musinsa_review_pants_1.csv')
df_pants2 = pd.read_csv('./musinsa_review_pants_2.csv')

In [136]:
print(df_pants1.shape, df_pants2.shape)

(558110, 19) (946857, 19)


In [137]:
df_pants = pd.concat([df_pants1, df_pants2], axis=0)
df_pants.shape

(1504967, 19)

In [138]:
df_pants

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946852,마산동호랭이,2020.08.10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
946853,~TOP~,2020.08.10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
946854,똘똘똘,2020.08.07,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
946855,브론즈_910d82e626b,2020.08.05,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [139]:
df_pants.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [140]:
df_pants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504967 entries, 0 to 946856
Data columns (total 19 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   userName       1503404 non-null  object
 1   date           1504967 non-null  object
 2   goodsNo        1504967 non-null  int64 
 3   userSexMen     1504967 non-null  int64 
 4   userSexWomen   1504967 non-null  int64 
 5   userHeight     1504967 non-null  object
 6   userWeight     1504967 non-null  object
 7   goodsSize      1504967 non-null  object
 8   reviewContent  1504967 non-null  object
 9   reviewImg      434510 non-null   object
 10  reviewStyle    1504967 non-null  int64 
 11  size           1504967 non-null  int64 
 12  bright         1504967 non-null  int64 
 13  color          1504967 non-null  int64 
 14  thickness      1504967 non-null  int64 
 15  weightness     1504967 non-null  int64 
 16  touch          1504967 non-null  int64 
 17  helpNo         1504967 non-n

In [141]:
df_pants.shape

(1504967, 19)

### 탈퇴 인원 제거

In [142]:
df_pants.dropna(subset=["userName"], axis=0, inplace=True)
df_pants = df_pants.reset_index(drop=True)

In [143]:
df_pants.shape

(1503404, 19)

### Date를 년, 월, 일로 등분

In [144]:
df_pants.dropna(subset=["date"], axis=0, inplace=True)

In [145]:
df_pants.shape

(1503404, 19)

In [146]:
df_pants.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


### goodsNo

In [147]:
df_pants.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [148]:
df_pants.shape

(1503404, 19)

### 성별

In [149]:
df_pants[df_pants.userSexMen == 1].shape

(1289057, 19)

In [150]:
df_pants[df_pants.userSexWomen==1].shape

(214191, 19)

In [151]:
a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\1405742730.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)


[7171,
 18542,
 25500,
 36751,
 47062,
 71306,
 81862,
 99514,
 99554,
 111520,
 113794,
 118877,
 130966,
 132652,
 138269,
 160093,
 164777,
 183899,
 205717,
 205720,
 220326,
 224275,
 242825,
 252431,
 252905,
 252942,
 261982,
 262476,
 263052,
 294324,
 327051,
 331165,
 347768,
 347822,
 390972,
 428956,
 429008,
 441682,
 459069,
 463433,
 510674,
 510696,
 512850,
 512872,
 523450,
 523565,
 552769,
 556256,
 561145,
 565234,
 579109,
 586481,
 598967,
 598975,
 600264,
 615534,
 639162,
 644467,
 660585,
 661990,
 667649,
 684220,
 685896,
 685904,
 690435,
 693422,
 707263,
 707273,
 707505,
 726545,
 726553,
 748025,
 748027,
 748789,
 750169,
 758905,
 758909,
 770338,
 771845,
 772430,
 775729,
 781779,
 804721,
 815861,
 832935,
 848221,
 848222,
 848277,
 851855,
 851880,
 851919,
 851945,
 872400,
 872430,
 872889,
 887390,
 892911,
 897508,
 911937,
 911955,
 924481,
 951355,
 962372,
 969276,
 1033484,
 1061233,
 1068316,
 1075208,
 1088355,
 1093537,
 1093538,
 110

In [152]:
df_pants = df_pants.drop(a)
df_pants.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503243,마산동호랭이,2020.08.10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503244,~TOP~,2020.08.10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503245,똘똘똘,2020.08.07,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503246,브론즈_910d82e626b,2020.08.05,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [153]:
df_pants.shape

(1503248, 19)

### 키와 몸무게

In [154]:
df_pants.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [155]:
df_pants.shape

(1503248, 19)

In [156]:
df_pants.userHeight = df_pants.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_pants.userHeight = df_pants.userHeight.astype('int64')

In [157]:
df_pants.userWeight = df_pants.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_pants.userWeight = df_pants.userWeight.astype('int64')

In [158]:
a = list((df_pants[df_pants.userHeight==0][df_pants.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\3922243988.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_pants[df_pants.userHeight==0][df_pants.userWeight==0]).index)


[1857,
 5977,
 41584,
 57173,
 93491,
 203467,
 208380,
 229242,
 263748,
 273395,
 327219,
 327529,
 354814,
 375034,
 523479,
 560404,
 562451,
 562452,
 611682,
 753052,
 787645,
 805291,
 861713,
 908501,
 908507,
 1061189,
 1136138,
 1143271,
 1179872,
 1258208,
 1381240,
 1404354,
 1408946,
 1431305,
 1440963,
 1467857]

In [159]:
df_pants = df_pants.drop(a)
df_pants.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503207,마산동호랭이,2020.08.10,1168907,1,0,178,84,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503208,~TOP~,2020.08.10,1168907,1,0,174,79,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503209,똘똘똘,2020.08.07,1168907,1,0,183,90,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503210,브론즈_910d82e626b,2020.08.05,1168907,1,0,175,76,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


### 이미지

In [160]:
df_pants.reviewStyle.value_counts()

0    1069286
1     433926
Name: reviewStyle, dtype: int64

In [161]:
df_pants.reviewImg = df_pants.reviewImg.fillna("-")

In [162]:
df_pants.reviewImg

0          https://image.msscdn.net/data/estimate/2258181...
1          https://image.msscdn.net/data/estimate/2258181...
2                                                          -
3                                                          -
4          https://image.msscdn.net/data/estimate/1281045...
                                 ...                        
1503399                                                    -
1503400                                                    -
1503401                                                    -
1503402                                                    -
1503403                                                    -
Name: reviewImg, Length: 1503212, dtype: object

### Contents

In [163]:
print(df_pants.reviewContent[0])
print(df_pants.reviewContent[1].split(","))

['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 그랬나바요', <br/>, '정말최고']
["['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱']"]


In [164]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_pants.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

18


In [165]:
result[1]

'[\'저한테는 너무 크다는 느낌 없이 M사이즈 선택에 후회 없을 베스트 핏으로 잘 맞았어요! 색감도 쨍하고 원단도 탄탄해서 맘에 들어요. 자연광에 잘 어울리는 색감이고 요즘 날씨에 편하게 입기 좋네요. 민트도 사고 싶어졌어요ㅠ 그리고 은근히 뒤에 로고 포인트가 디자인의 킬포입니다 전 앞면 보고 반해서 샀는데 뒷판도 너무 예뻐요. 에어포스에 신으면 완벽합니다>\', \'!</div\', \'\\n\', <div class="review-evaluation">\r\n<ul class="review-evaluation__list">\r\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\r\n<li class="review-evaluation__item">밝기 <span>밝아요</span></li>\r\n<li class="review-evaluation__item">색감 <span>선명해요</span></li>\r\n<li class="review-evaluation__item">두께감 <span>보통이에요</span></li>\r\n</ul>\r\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\r\n<div class="review-content-photo__wrap">\r\n<ul class="review-content-photo__list">\r\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1554661_0/gallery_608c4885ca283.jpg.list\');">\r\n<img alt="아카이브 볼드(ARCHIVE BOLD) 939 LOGO SWEAT PANTS (DEEP RED) 후기" src="//image.msscdn.net/d

In [166]:
for i in result:
    idx = df_pants[df_pants.reviewContent==i].index[0]
    df_pants.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [167]:
df_pants.reviewContent = df_pants.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("' ", "").replace("'", "").replace(", <br/>", "").split(","))

In [168]:
print(df_pants.reviewContent[17889])
print(df_pants.reviewContent[1])

['너무 예뻐서 회색 검은색 두 컬러 구매 완료>.', ' 핏 좋고 편하고 따수워용']
['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱']


### 사이즈

In [169]:
sub = df_pants[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(4701, 4701+len(sub))
sub.index = range(4701, 4701+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [170]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [171]:
len(sub)

74137

In [172]:
for i in range(4701, len(sub)+4701):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,2258181,4701
4702,2258181,L,2258181,4702
4703,1281045,S,1281045,4703
4704,2328543,L,2328543,4704
4705,2346636,블랙/XL,2346636,4705
...,...,...,...,...
78833,1168907,33,1168907,78833
78834,1168907,36,1168907,78834
78835,1168907,38,1168907,78835
78836,1168907,40,1168907,78836


In [173]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74137 entries, 4701 to 78837
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     74137 non-null  object
 1   goodsSize   74137 non-null  object
 2   newGoodsNo  74137 non-null  object
 3   index       74137 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [174]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,22581814701,4701
4702,2258181,L,22581814702,4702
4703,1281045,S,12810454703,4703
4704,2328543,L,23285434704,4704
4705,2346636,블랙/XL,23466364705,4705
...,...,...,...,...
78833,1168907,33,116890778833,78833
78834,1168907,36,116890778834,78834
78835,1168907,38,116890778835,78835
78836,1168907,40,116890778836,78836


In [175]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [176]:
df_pants = pd.merge(left=df_pants, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_pants.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,기무태,2022.01.10,2258181,1,0,171,74,M,"[세트별로 다사서 이번엔 바지만 샀는데, 이것도 위에 같이 살걸 그랬나바요, 정말최고]",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814701
1,10407,2021.12.24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814702
2,기무태,2022.01.10,2258181,1,0,171,74,M,"[진짜 너무 이뻐서바지만 또사요, 정말 최고 아이템]",-,0,2,2,2,2,0,0,0,0,22581814701
3,10407,2021.12.24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],-,0,2,2,2,2,0,0,0,0,22581814702
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173,69,S,"[제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0,12810454703


In [177]:
df_pants.shape

(1503212, 20)

In [178]:
df_pants.insert(0, 'reviewId', 0, True)

In [179]:
df_pants.reviewId = range(20822, 20822 + len(df_pants))

In [180]:
df_pants

,reviewId,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,20822,기무태,2022.01.10,2258181,1,0,171,74,M,"[세트별로 다사서 이번엔 바지만 샀는데, 이것도 위에 같이 살걸 그랬나바요, 정말최고]",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814701
1,20823,10407,2021.12.24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814702
2,20824,기무태,2022.01.10,2258181,1,0,171,74,M,"[진짜 너무 이뻐서바지만 또사요, 정말 최고 아이템]",-,0,2,2,2,2,0,0,0,0,22581814701
3,20825,10407,2021.12.24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],-,0,2,2,2,2,0,0,0,0,22581814702
4,20826,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173,69,S,"[제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0,12810454703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503207,1524029,마산동호랭이,2020.08.10,1168907,1,0,178,84,34,"[핏 좋습니다, 살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다]",-,0,2,2,2,2,0,0,0,0,116890778829
1503208,1524030,~TOP~,2020.08.10,1168907,1,0,174,79,32,[상품 잘 받았습니다 배송도 빠르게 잘 받았습니다],-,0,2,2,2,2,0,0,0,0,116890778831
1503209,1524031,똘똘똘,2020.08.07,1168907,1,0,183,90,36,[가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다],-,0,2,2,2,2,0,0,0,0,116890778834
1503210,1524032,브론즈_910d82e626b,2020.08.05,1168907,1,0,175,76,32,"[32인데 허리랑 허벅지가 좀 타이트해요, 길이는 적당히 깁니다]",-,0,2,2,2,1,0,0,0,0,116890778831


In [181]:
df_pants.to_json('review_pants.json', orient='records', force_ascii=False)

# Outer 리뷰 전처리

In [182]:
df_outer1 = pd.read_csv('./musinsa_review_outer_1.csv')
df_outer2 = pd.read_csv('./musinsa_review_outer_2.csv')

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [183]:
print(df_outer1.shape, df_outer2.shape)

(530159, 19) (343280, 19)


In [184]:
df_outer = pd.concat([df_outer1, df_outer2], axis=0)
df_outer.shape

(873439, 19)

In [185]:
df_outer

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180cm,61kg,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022.03.15,2391602,0,1,165cm,46kg,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021.07.11,1962220,0,1,163cm,54kg,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022.02.20,2252652,1,0,175cm,78kg,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021.12.10,1549580,1,0,178cm,90kg,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343275,좡좡쓰,2020.12.20,1653581,0,1,162cm,50kg,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
343276,좡좡쓰,2020.12.11,1653581,1,0,177cm,64kg,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
343277,연현흠,2020.12.04,1653581,1,0,173cm,65kg,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
343278,뉴비_e8a414c8,2020.11.12,1653581,1,0,175cm,78kg,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [186]:
df_outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 873439 entries, 0 to 343279
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   userName       872653 non-null  object 
 1   date           873439 non-null  object 
 2   goodsNo        873439 non-null  object 
 3   userSexMen     873439 non-null  object 
 4   userSexWomen   873439 non-null  object 
 5   userHeight     873438 non-null  object 
 6   userWeight     873439 non-null  object 
 7   goodsSize      873439 non-null  object 
 8   reviewContent  873439 non-null  object 
 9   reviewImg      245815 non-null  object 
 10  reviewStyle    873439 non-null  int64  
 11  size           873439 non-null  int64  
 12  bright         873439 non-null  int64  
 13  color          873439 non-null  int64  
 14  thickness      873439 non-null  int64  
 15  weightness     873438 non-null  float64
 16  touch          873438 non-null  float64
 17  helpNo         873438 non-nul

In [187]:
df_outer.shape

(873439, 19)

### 탈퇴 인원 제거

In [188]:
df_outer.dropna(subset=["userName"], axis=0, inplace=True)
df_outer = df_outer.reset_index(drop=True)

In [189]:
df_outer.shape

(872653, 19)

### Date를 년, 월, 일로 등분

In [190]:
df_outer.dropna(subset=["date"], axis=0, inplace=True)

In [191]:
df_outer.shape

(872653, 19)

In [192]:
df_outer.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180cm,61kg,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022.03.15,2391602,0,1,165cm,46kg,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021.07.11,1962220,0,1,163cm,54kg,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022.02.20,2252652,1,0,175cm,78kg,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021.12.10,1549580,1,0,178cm,90kg,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0


### goodsNo

In [193]:
df_outer.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [194]:
df_outer.shape

(872653, 19)

In [195]:
df_outer.goodsNo = df_outer.goodsNo.astype('int')

ValueError: invalid literal for int() with base 10: ' 56kg'

In [ ]:
df_outer.info()

In [ ]:
df_outer[df_outer.userSexMen=='IV/095']

In [ ]:
df_outer.userSexMen.unique()

### 유저 성별

In [ ]:
df_outer.userSexMen = df_outer.userSexMen.astype('int')
df_outer.userSexWomen = df_outer.userSexWomen.astype('int')

In [ ]:
df_outer[df_outer.userSexMen == 1].shape

In [ ]:
df_outer[df_outer.userSexWomen==1].shape

In [ ]:
a = list((df_outer[df_outer.userSexMen==0][df_outer.userSexWomen==0]).index)
a

In [ ]:
df_outer = df_outer.drop(a)
df_outer.reset_index(drop=True)

In [196]:
df_outer.shape

(872653, 19)

In [197]:
872628

872628

### 키와 몸무게

In [198]:
df_outer.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [199]:
df_outer.shape

(872652, 19)

In [204]:
df_outer.userHeight.unique()

array(['180', '165', '163', '175', '178', '177', '162', '176', '160',
       '170', '182', '183', '167', '173', '158', '155', '189', '174',
       '172', '164', '179', '168', '181', '169', '166', '171', '154',
       '153', '186', '157', '188', '184', '185', '156', '161', '200',
       '152', '149', '159', '191', '151', '187', '192', '144', '147',
       '150', '209', '190', '146', '110', '193', '100', '148', '194',
       '199', '111', '220', '120', '145', '210', '0', '123', '1560',
       '195', '112', '133', '140', '142', '197', '196', '99', '138',
       '130', '300', '208', '213', '202', '217', '143', '198', '104',
       '108', '1', '117', '127', '1774', '17', '116', '14', '124', '1730',
       '70', '74', '280', '201', '102', '207', '18', '203', '212', '500',
       '103', '<br/>', '119', '82523', '105', '135', '137', '205', '999',
       '71', '204', '1172'], dtype=object)

In [211]:
df_outer[df_outer.userHeight=='<br/>']

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
528773,)',<br/>,'디자인도 마음에 들고 겉 제질이 스포티지해서',<br/>,'어디에 매치해도 잘 어울리는거 같아요',<br/>,'상품이 생각보다 가벼워서 놀라고 가벼운데 따뜻해서 ',<br/>,"'두번 놀랐네요 아주우 맘에 들어요 :)']""",https://image.msscdn.net/data/estimate/1686781...,1,2,2,2,0,1.0,0.0,0.0,0.0


In [213]:
df_outer = df_outer.drop([528773])

In [214]:
df_outer.userHeight = df_outer.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_outer.userHeight = df_outer.userHeight.astype('int64')

In [215]:
df_outer.userWeight = df_outer.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_outer.userWeight = df_outer.userWeight.astype('int64')

In [216]:
a = list((df_outer[df_outer.userHeight==0][df_outer.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\2122965503.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_outer[df_outer.userHeight==0][df_outer.userWeight==0]).index)


[16530,
 205413,
 245163,
 250427,
 304380,
 325996,
 328326,
 333749,
 333853,
 438349,
 466564,
 509047,
 509334,
 577679,
 628400,
 758477,
 765167,
 768593,
 798887,
 804272,
 805940,
 856238,
 865576,
 865577,
 865581]

In [217]:
df_outer = df_outer.drop(a)
df_outer.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180,61,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022.03.15,2391602,0,1,165,46,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021.07.11,1962220,0,1,163,54,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022.02.20,2252652,1,0,175,78,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021.12.10,1549580,1,0,178,90,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872621,좡좡쓰,2020.12.20,1653581,0,1,162,50,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872622,좡좡쓰,2020.12.11,1653581,1,0,177,64,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872623,연현흠,2020.12.04,1653581,1,0,173,65,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
872624,뉴비_e8a414c8,2020.11.12,1653581,1,0,175,78,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [218]:
df_outer.shape

(872626, 19)

### 이미지

In [219]:
df_outer.reviewStyle.value_counts()

0    627018
1    245608
Name: reviewStyle, dtype: int64

In [220]:
df_outer.reviewImg = df_outer.reviewImg.fillna("-")

In [221]:
df_outer.reviewImg

0         https://image.msscdn.net/data/estimate/1766979...
1                                                         -
2                                                         -
3                                                         -
4         https://image.msscdn.net/data/estimate/1549580...
                                ...                        
872648                                                    -
872649                                                    -
872650                                                    -
872651                                                    -
872652                                                    -
Name: reviewImg, Length: 872626, dtype: object

### Contents

In [222]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1].split(","))

['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 딱 적당합니다.', <br/>, <br/>, '안 두꺼운데 얇은 편이라 가을과 봄에 입으면 딱 좋고', <br/>, <br/>, '교복같은 무슨 옷이던 잘 어울려서 더 좋은거 같아요.', <br/>, <br/>, '디자인이 과하고 좀 튈거라 생각했는데 ', <br/>, '과하지 않고 깔끔해요.', <br/>, <br/>, '단점은 핏이 조금 깔끔하지 못할때도 있어요.']
["['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요']"]


In [223]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_outer.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

21


In [224]:
for i in result:
    idx = df_outer[df_outer.reviewContent==i].index[0]
    df_outer.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [225]:
df_outer.reviewContent = df_outer.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [226]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1])

['제가 어두운 옷보단 밝은 옷을 좋아하는데', '너무 밝지도 않고 딱 적당합니다.', '안 두꺼운데 얇은 편이라 가을과 봄에 입으면 딱 좋고', '교복같은 무슨 옷이던 잘 어울려서 더 좋은거 같아요.', '디자인이 과하고 좀 튈거라 생각했는데', '과하지 않고 깔끔해요.', '단점은 핏이 조금 깔끔하지 못할때도 있어요.']
['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요']


In [227]:
df_outer.shape

(872626, 19)

### 사이즈

In [228]:
sub = df_outer[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(78839, 78839+len(sub))
sub.index = range(78839, 78839+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [229]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [230]:
len(sub)

52546

In [231]:
for i in range(78839, len(sub)+78839):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
78839,1766979,L,1766979,78839
78840,2391602,FREE,2391602,78840
78841,1962220,black/free,1962220,78841
78842,2252652,레드/L,2252652,78842
78843,1549580,FREE,1549580,78843
...,...,...,...,...
131380,1229923,XL,1229923,131380
131381,2107087,FREE,2107087,131381
131382,1653581,M,1653581,131382
131383,1653581,L,1653581,131383


In [232]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52546 entries, 78839 to 131384
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     52546 non-null  object
 1   goodsSize   52546 non-null  object
 2   newGoodsNo  52546 non-null  object
 3   index       52546 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [233]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub.index = range(len(sub))

In [234]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [235]:
df_outer_sub = pd.merge(left=df_outer, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_outer_sub

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180,61,L,"[제가 어두운 옷보단 밝은 옷을 좋아하는데, 너무 밝지도 않고 딱 적당합니다., 안...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0,1.766980e+11
1,무신가신사,2022.03.15,2391602,0,1,165,46,FREE,[예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요],-,0,2,2,2,2,0.0,2.0,0.0,0.0,2.391603e+11
2,뉴비_3c17ca57,2021.07.11,1962220,0,1,163,54,black/free,[재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하려구요],-,0,2,2,2,2,0.0,2.0,0.0,0.0,1.962221e+11
3,푸쳐핸섭,2022.02.20,2252652,1,0,175,78,레드/L,"[빨간색 좋아해서 삿는뎅, 맘에는 드는뎅, 구명조끼 같다는 동료의 말에 빵터짐, 필...",-,0,2,2,1,1,0.0,0.0,0.0,0.0,2.252653e+11
4,mmd99,2021.12.10,1549580,1,0,178,90,FREE,[이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0,1.549581e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911583,좡좡쓰,2020.12.20,1653581,0,1,162,50,M,[나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여],-,0,2,2,2,2,0.0,0.0,0.0,0.0,1.653581e+12
911584,좡좡쓰,2020.12.11,1653581,1,0,177,64,L,[이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.],-,0,2,2,2,2,0.0,0.0,0.0,0.0,1.653581e+12
911585,연현흠,2020.12.04,1653581,1,0,173,65,L,"[한겨울에 입어도 될 정도로 뜨뜻~합니다, 색감도 좋아요]",-,0,2,1,1,1,0.0,0.0,0.0,0.0,1.653581e+12
911586,뉴비_e8a414c8,2020.11.12,1653581,1,0,175,78,L,[두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯],-,0,1,2,2,1,0.0,0.0,0.0,0.0,1.653581e+12


In [236]:
df_outer.insert(0, 'reviewId', 0, True)

In [237]:
df_outer.reviewId = range(1524034, 1524034 + len(df_outer))

In [238]:
df_outer

,reviewId,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,1524034,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180,61,L,"[제가 어두운 옷보단 밝은 옷을 좋아하는데, 너무 밝지도 않고 딱 적당합니다., 안...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,1524035,무신가신사,2022.03.15,2391602,0,1,165,46,FREE,[예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요],-,0,2,2,2,2,0.0,2.0,0.0,0.0
2,1524036,뉴비_3c17ca57,2021.07.11,1962220,0,1,163,54,black/free,[재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하려구요],-,0,2,2,2,2,0.0,2.0,0.0,0.0
3,1524037,푸쳐핸섭,2022.02.20,2252652,1,0,175,78,레드/L,"[빨간색 좋아해서 삿는뎅, 맘에는 드는뎅, 구명조끼 같다는 동료의 말에 빵터짐, 필...",-,0,2,2,1,1,0.0,0.0,0.0,0.0
4,1524038,mmd99,2021.12.10,1549580,1,0,178,90,FREE,[이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872648,2396655,좡좡쓰,2020.12.20,1653581,0,1,162,50,M,[나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여],-,0,2,2,2,2,0.0,0.0,0.0,0.0
872649,2396656,좡좡쓰,2020.12.11,1653581,1,0,177,64,L,[이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.],-,0,2,2,2,2,0.0,0.0,0.0,0.0
872650,2396657,연현흠,2020.12.04,1653581,1,0,173,65,L,"[한겨울에 입어도 될 정도로 뜨뜻~합니다, 색감도 좋아요]",-,0,2,1,1,1,0.0,0.0,0.0,0.0
872651,2396658,뉴비_e8a414c8,2020.11.12,1653581,1,0,175,78,L,[두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯],-,0,1,2,2,1,0.0,0.0,0.0,0.0


In [239]:
df_outer.to_json('review_outer.json', orient='records', force_ascii=False)

# TOP 리뷰 전처리

In [240]:
df_top1_1 = pd.read_csv('./musinsa_review_top1_1.csv')
df_top1_2 = pd.read_csv('./musinsa_review_top1_2.csv', encoding='cp949')
df_top1_3 = pd.read_csv('./musinsa_review_top1_3.csv', encoding='cp949')
df_top2_1 = pd.read_csv('./musinsa_review_top2_1.csv')
df_top2_2 = pd.read_csv('./musinsa_review_top2_2.csv')
df_top2_3 = pd.read_csv('./musinsa_review_top2_3.csv')
df_top3_1 = pd.read_csv('./musinsa_review_top3_1.csv')
df_top3_2 = pd.read_csv('./musinsa_review_top3_2.csv')
df_top3_3 = pd.read_csv('./musinsa_review_top3_3.csv')
df_top3_4 = pd.read_csv('./musinsa_review_top3_4.csv')
df_top3_5 = pd.read_csv('./musinsa_review_top3_5.csv')
df_top4_1 = pd.read_csv('./musinsa_review_top4_1.csv')
df_top4_2 = pd.read_csv('./musinsa_review_top4_2.csv')

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [241]:
print(df_top1_1.shape, df_top1_2.shape, df_top1_3.shape, df_top2_1.shape, df_top2_2.shape, df_top2_3.shape, df_top3_1.shape, df_top3_2.shape, df_top3_3.shape, df_top3_4.shape, df_top3_5.shape, df_top4_1.shape, df_top4_2.shape)

(282510, 19) (137842, 19) (209770, 19) (141551, 19) (174419, 19) (48441, 19) (70205, 19) (56159, 19) (59067, 19) (59259, 19) (90934, 19) (457875, 19) (14749, 19)


In [242]:
282510+137842+209770+141551+174419+48441+70205+56159+59067+59259+90934+457875+14749

1802781

In [243]:
df_top = pd.concat([df_top1_1, df_top1_2, df_top1_3, df_top2_1, df_top2_2, df_top2_3, df_top3_1, df_top3_2, df_top3_3, df_top3_4, df_top3_5, df_top4_1, df_top4_2], axis=0)
df_top.shape

(1802781, 19)

In [244]:
df_top

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14744,뉴비_e8fb6ff,2022.03.04,2331506,1,0,177cm,74kg,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
14745,오이파리,2022.03.13,2331511,1,0,180cm,70kg,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
14746,불꽃비형,2022.03.07,2331511,1,0,171cm,70kg,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
14747,goooooood!,2022.02.27,2331511,1,0,171cm,65kg,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [245]:
df_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1802781 entries, 0 to 14748
Data columns (total 19 columns):
 #   Column         Dtype  
---  ------         -----  
 0   userName       object 
 1   date           object 
 2   goodsNo        object 
 3   userSexMen     object 
 4   userSexWomen   object 
 5   userHeight     object 
 6   userWeight     object 
 7   goodsSize      object 
 8   reviewContent  object 
 9   reviewImg      object 
 10  reviewStyle    float64
 11  size           float64
 12  bright         float64
 13  color          float64
 14  thickness      float64
 15  weightness     float64
 16  touch          float64
 17  helpNo         float64
 18  styleLikeNo    float64
dtypes: float64(9), object(10)
memory usage: 275.1+ MB


In [246]:
df_top = df_top.drop([42062])

In [247]:
df_top.goodsNo = df_top.goodsNo.astype('int')

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [248]:
df_top.shape

(1802769, 19)

### 탈퇴 인원 제거

In [249]:
df_top.dropna(subset=["userName"], axis=0, inplace=True)
df_top = df_top.reset_index(drop=True)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [250]:
df_top.shape

(1800956, 19)

### Date를 년, 월, 일로 등분

In [251]:
df_top.dropna(subset=["date"], axis=0, inplace=True)

In [252]:
df_top.shape

(1800956, 19)

In [253]:
df_top.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


### goodsNo

In [254]:
df_top.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [255]:
df_top.shape

(1800956, 19)

### 유저 성별

In [256]:
df_top.userSexMen = df_top.userSexMen.astype('int')
df_top.userSexWomen = df_top.userSexWomen.astype('int')

In [257]:
df_top[df_top.userSexMen == 1].shape

(1317618, 19)

In [258]:
df_top[df_top.userSexWomen==1].shape

(483214, 19)

In [259]:
a = list((df_top[df_top.userSexMen==0][df_top.userSexWomen==0]).index)
len(a)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\2693055301.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_top[df_top.userSexMen==0][df_top.userSexWomen==0]).index)


120

In [260]:
df_top = df_top.drop(a)
df_top.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800831,뉴비_e8fb6ff,2022.03.04,2331506,1,0,177cm,74kg,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800832,오이파리,2022.03.13,2331511,1,0,180cm,70kg,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1800833,불꽃비형,2022.03.07,2331511,1,0,171cm,70kg,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800834,goooooood!,2022.02.27,2331511,1,0,171cm,65kg,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [261]:
df_top.shape

(1800836, 19)

### 키와 몸무게

In [262]:
df_top.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [263]:
df_top.shape

(1800836, 19)

In [264]:
df_top.userHeight = df_top.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_top.userHeight = df_top.userHeight.astype('int64')

In [265]:
df_top.userWeight = df_top.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_top.userWeight = df_top.userWeight.astype('int64')

In [266]:
a = list((df_top[df_top.userHeight==0][df_top.userWeight==0]).index)
len(a)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3012\3749120873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_top[df_top.userHeight==0][df_top.userWeight==0]).index)


97

In [267]:
df_top = df_top.drop(a)
df_top.reset_index(drop=True)

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,2062047,0,1,168,67,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,2062047,0,1,154,50,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,2062047,0,1,171,54,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800734,뉴비_e8fb6ff,2022.03.04,2331506,1,0,177,74,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800735,오이파리,2022.03.13,2331511,1,0,180,70,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1800736,불꽃비형,2022.03.07,2331511,1,0,171,70,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800737,goooooood!,2022.02.27,2331511,1,0,171,65,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [268]:
df_top.shape

(1800739, 19)

### 이미지

In [269]:
df_top.reviewStyle.value_counts()

0.0    1386139
1.0     414599
Name: reviewStyle, dtype: int64

In [270]:
df_top.reviewImg = df_top.reviewImg.fillna("-")

In [271]:
df_top.reviewImg

0                                                          -
1                                                          -
2                                                          -
3          https://image.msscdn.net/data/estimate/2085464...
4                                                          -
                                 ...                        
1800951                                                    -
1800952                                                    -
1800953                                                    -
1800954                                                    -
1800955                                                    -
Name: reviewImg, Length: 1800739, dtype: object

### Contents

In [272]:
print(df_top.reviewContent[0])
print(df_top.reviewContent[1].split(","))

['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오버핏으로 잘 맞을줄알았으나 정핏으로맞아오']
["['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.']"]


In [273]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_top.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

34


In [274]:
for i in result:
    idx = df_top[df_top.reviewContent==i].index[0]
    df_top.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [275]:
df_top.reviewContent = df_top.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [276]:
print(df_top.reviewContent[0])
print(df_top.reviewContent[1])

['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오버핏으로 잘 맞을줄알았으나 정핏으로맞아오']
['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.']


### 사이즈

In [277]:
sub = df_top[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(131385, 131385+len(sub))
sub.index = range(131385, 131385+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [278]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [279]:
len(sub)

150221

In [280]:
for i in range(131385, len(sub)+131385):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
131385,2062047,1(ONESIZE),2062047,131385
131386,2085464,IVORY/M,2085464,131386
131387,2085464,IVORY/S,2085464,131387
131388,1984454,L,1984454,131388
131389,2287021,L,2287021,131389
...,...,...,...,...
281601,2331508,105,2331508,281601
281602,2331508,100,2331508,281602
281603,2331506,100,2331506,281603
281604,2331511,100,2331511,281604


In [281]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150221 entries, 131385 to 281605
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   goodsNo     150221 non-null  object
 1   goodsSize   150221 non-null  object
 2   newGoodsNo  150221 non-null  object
 3   index       150221 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [282]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
131385,2062047,1(ONESIZE),2062047131385,131385
131386,2085464,IVORY/M,2085464131386,131386
131387,2085464,IVORY/S,2085464131387,131387
131388,1984454,L,1984454131388,131388
131389,2287021,L,2287021131389,131389
...,...,...,...,...
281601,2331508,105,2331508281601,281601
281602,2331508,100,2331508281602,281602
281603,2331506,100,2331506281603,281603
281604,2331511,100,2331511281604,281604


In [283]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [284]:
df_top = pd.merge(left=df_top, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_top.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,sbom,2022.01.04,2062047,0,1,168,67,1(ONESIZE),[음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오...,-,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,2062047131385
1,mishoww,2021.11.19,2062047,0,1,154,50,1(ONESIZE),[길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.],-,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2062047131385
2,저리치워,2021.08.27,2062047,0,1,171,54,1(ONESIZE),"[프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌, , , ]",-,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2062047131385
3,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,[안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2085464131386
4,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,[안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요],-,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2085464131386


In [285]:
df_top.shape

(1800739, 20)

In [286]:
sum(df_top.newGoodsNo.isna())

0

In [287]:
df_top.insert(0, 'reviewId', 0, True)

In [288]:
df_top.reviewId = range(2396660, 2396660 + len(df_top))

In [289]:
df_top

,reviewId,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,2396660,sbom,2022.01.04,2062047,0,1,168,67,1(ONESIZE),[음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오...,-,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,2062047131385
1,2396661,mishoww,2021.11.19,2062047,0,1,154,50,1(ONESIZE),[길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.],-,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2062047131385
2,2396662,저리치워,2021.08.27,2062047,0,1,171,54,1(ONESIZE),"[프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌, , , ]",-,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2062047131385
3,2396663,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,[안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2085464131386
4,2396664,하아안빈,2022.01.10,2085464,1,0,171,60,IVORY/M,[안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요],-,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2085464131386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800734,4197394,뉴비_e8fb6ff,2022.03.04,2331506,1,0,177,74,100,[이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습니다!],-,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2331506281603
1800735,4197395,오이파리,2022.03.13,2331511,1,0,180,70,100,"[두껍지않아서 지금입기 딱 좋아요, 정사이즈 넉넉하게 맞습니다]",-,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,2331511281604
1800736,4197396,불꽃비형,2022.03.07,2331511,1,0,171,70,095,[컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......],-,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2331511281605
1800737,4197397,goooooood!,2022.02.27,2331511,1,0,171,65,095,"[정사이즈 ~세미오버핏 중간 사이즈입니다~~, 색감 쨍하니 좋습니다]",-,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2331511281605


In [290]:
df_top.to_json('review_top.json', orient='records', force_ascii=False)